<a href="https://colab.research.google.com/github/ryannp-cerebras/test/blob/main/Build_your_own_Perplexity_%7C_Cerebras_and_Exa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Build your own Perplexity | Cerebras and Exa

Today, we'll build a Perplexity-style deep research assistant that can automatically search the web, analyzes multiple sources, and provide structured insights in under 60 seconds.

Try out the full deep research chat [here](https://cerebras-deepresearch.netlify.app).



For this workshop, you'll need:
- Cerebras API: the fastest inference provider, [get started for free here](https://cloud.cerebras.ai?utm_source=exademo)
- Exa API: The search engine for AI, [get started for free here](https://exa.ai/?utm_source=cerebras-research)

If you have any questions, please reach out on the [Cerebras Discord](https://cerebras.ai/discord)

## Step #1: Environment Setup
First, let's install all the necessary libraries, import everything we need, and configure our API credentials.

In [ ]:
%pip install exa-py cerebras-cloud-sdk

from exa_py import Exa
from cerebras.cloud.sdk import Cerebras

# Add your API keys here
EXA_API_KEY = ""
CEREBRAS_API_KEY = ""

client = Cerebras(api_key = CEREBRAS_API_KEY)
exa = Exa(api_key = EXA_API_KEY)

print("✅ Setup complete")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 5.6 MB/s eta 0:00:00
✅ Setup complete


## Step #2: Web Search Function

Our first core function handles web searching using Exa's auto search. This is advantageous because it uses a blend of keyword and neural search to find both exact matches and semantic similarities. It also returns the content of each scraped URL.

In [ ]:
def search_web(query, num=5):
    """Search the web using Exa's auto search"""
    result = exa.search_and_contents(
      query,
      type = "auto",
      num_results = num,
      text={"max_characters": 1000}
    )
    return result.results

print("✅ Search function ready")

✅ Search function ready


# Step #3: AI Analysis Function

This function leverages Cerebras fast inference to analyze content and generate insights. We'll use it both for structured JSON responses and regular text analysis throughout our research process.

In [ ]:
def ask_ai(prompt):
    """Get AI response from Cerebras"""
    chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="llama-4-scout-17b-16e-instruct",
      max_tokens = 600,
      temperature = 0.2
    )
    return chat_completion.choices[0].message.content

print("✅ AI function ready")

✅ AI function ready


#Step #4: Research Function

Now we'll build our research methodology. The first part of the below cell performs the initial search and gathers our first batch of sources, just like when you first query Perplexity.

Then, the AI is queried to generate a conclusion based on the source data.

In [ ]:
  def research_topic(query):
    """Main research function that orchestrates the entire process"""
    print(f"🔍 Researching: {query}")

    # Search for sources
    results = search_web(query, 5)
    print(f"📊 Found {len(results)} sources")

    # Get content from sources
    sources = []
    for result in results:
        content = result.text
        title = result.title
        if content and len(content) > 200:
            sources.append({
                "title": title,
                "content": content
            })

    print(f"📄 Scraped {len(sources)} sources")

    if not sources:
        return {"summary": "No sources found", "insights": []}

    # Create context for AI analysis
    context = f"Research query: {query}\n\nSources:\n"
    for i, source in enumerate(sources[:4], 1):
        context += f"{i}. {source['title']}: {source['content'][:400]}...\n\n"
        # ^^ get rid of this to use API params!
        # best practices - https://www.anthropic.com/engineering/built-multi-agent-research-system

    # Ask AI to analyze and synthesize
    prompt = f"""{context}

Based on these sources, provide:
1. A comprehensive summary (2-3 sentences)
2. Three key insights as bullet points

Format your response exactly like this:
SUMMARY: [your summary here]

INSIGHTS:
- [insight 1]
- [insight 2]
- [insight 3]"""

    response = ask_ai(prompt)
    print("🧠 Analysis complete")

    return {"query": query, "sources": len(sources), "response": response}

print("✅ Research function ready")

✅ Research function ready


#Step #5: Add Research Depth

Now let's make our research **intelligent** instead of just searching once and hoping for the best.

Here's the problem with basic search: You ask "What are the latest AI breakthroughs?" and get random articles. But what if those articles all focus on ChatGPT and miss robotics? You'd never know what you're missing.

Here is a more advanced research flow.

```
You ask: "What's driving the new wave of AI agents?"
 ↓
Layer 1: Broad search finds 6 sources about agent frameworks like AutoGPT, LangChain, and OpenAI’s GPT-4o
 ↓
AI reads them and thinks: “These all focus on software… but what’s enabling real-time speed?”
 ↓
Layer 2: Targeted search for "AI hardware for real-time agents" and "fast inference for LLMs"
 ↓
Final synthesis: Combines insights on agent software + breakthroughs in inference speed (Cerebras, NVIDIA, etc) =
A complete picture of what powers real-time AI agents today
```

In [ ]:
def deeper_research_topic(query):
    """Two-layer research for better depth"""
    print(f"🔍 Researching: {query}")

    # Layer 1: Initial search
    results = search_web(query, 6)
    sources = []
    for result in results:
        if result.text and len(result.text) > 200:
            sources.append({"title": result.title, "content": result.text})

    print(f"Layer 1: Found {len(sources)} sources")

    if not sources:
        return {"summary": "No sources found", "insights": []}

    # Get initial analysis and identify follow-up topic
    context1 = f"Research query: {query}\n\nSources:\n"
    for i, source in enumerate(sources[:4], 1):
        context1 += f"{i}. {source['title']}: {source['content'][:300]}...\n\n"

    follow_up_prompt = f"""{context1}

Based on these sources, what's the most important follow-up question that would deepen our understanding of "{query}"?

Respond with just a specific search query (no explanation):"""

    follow_up_query = ask_ai(follow_up_prompt).strip().strip('"')

    # Layer 2: Follow-up search
    print(f"Layer 2: Investigating '{follow_up_query}'")
    follow_results = search_web(follow_up_query, 4)

    for result in follow_results:
        if result.text and len(result.text) > 200:
            sources.append({"title": f"[Follow-up] {result.title}", "content": result.text})

    print(f"Total sources: {len(sources)}")

    # Final synthesis
    all_context = f"Research query: {query}\nFollow-up: {follow_up_query}\n\nAll Sources:\n"
    for i, source in enumerate(sources[:7], 1):
        all_context += f"{i}. {source['title']}: {source['content'][:300]}...\n\n"

    final_prompt = f"""{all_context}

Provide a comprehensive analysis:

SUMMARY: [3-4 sentences covering key findings from both research layers]

INSIGHTS:
- [insight 1]
- [insight 2]
- [insight 3]
- [insight 4]

DEPTH GAINED: [1 sentence on how the follow-up search enhanced understanding]"""

    response = ask_ai(final_prompt)
    return {"query": query, "sources": len(sources), "response": response}

print("✅ Enhanced research function ready")

✅ Enhanced research function ready


In [ ]:
# Test the enhanced research system
result = deeper_research_topic("climate change solutions 2025")

# Display results
print("\n" + "="*50)
print("ENHANCED RESEARCH RESULTS")
print("="*50)
print(f"Query: {result['query']}")
print(f"Sources analyzed: {result['sources']}")
print(f"\n{result['response']}")
print("="*50)

# Try more topics
print("\nTry these:")
print("deeper_research_topic('quantum computing advances')")
print("deeper_research_topic('space exploration news')")

🔍 Researching: climate change solutions 2025
Layer 1: Found 6 sources
Layer 2: Investigating 'What are the top 3 climate change solutions proposed for 2025 by MIT Solve, UNDP, and UNFCCC?'
Total sources: 10

ENHANCED RESEARCH RESULTS
Query: climate change solutions 2025
Sources analyzed: 10

**SUMMARY**

The research query on climate change solutions for 2025 revealed various initiatives and proposals from prominent organizations. MIT Solve, UNDP, and UNFCCC are actively seeking and promoting innovative solutions to address climate change. The top 3 climate change solutions proposed for 2025 by these organizations focus on creating a zero-carbon world, adapting to a warming climate, and reducing greenhouse gas emissions. These solutions include leveraging technology, such as AI and graphene, to capture carbon and promote sustainable development.

**INSIGHTS**

* **MIT Solve's 2025 Global Climate Challenge**: The challenge focuses on creating a zero-carbon world and adapting to a warmin

# Step #6 (Optional): Anthropic Multi-Agent Research

What makes [Anthropic's approach](https://www.anthropic.com/engineering/built-multi-agent-research-system) special?

It uses intelligent orchestration with specialized agents working in parallel.

Usually, one AI does everything sequentially
- Search → analyze → search → analyze (slow, limited)

Anthropic's approach is to allow a lead agent delegate to specialized subagents
- Lead agent breaks down "AI safety research" into:
  - Subagent 1: Current AI safety techniques  
  - Subagent 2: Recent regulatory developments
  - Subagent 3: Industry implementation challenges
- All agents work simultaneously = 3x faster, better coverage

The result is parallel intelligence that scales to complex research tasks.

In [ ]:
def anthropic_multiagent_research(query):
    """
    Simple implementation of Anthropic's multi-agent approach:
    1. Lead agent plans and delegates
    2. Specialized subagents work in parallel
    3. Lead agent synthesizes results
    """
    print(f"🤖 Anthropic Multi-Agent Research: {query}")
    print("-" * 50)

    # Step 1: Lead Agent - Task Decomposition & Delegation
    print("👨‍💼 LEAD AGENT: Planning and delegating...")

    delegation_prompt = f"""You are a Lead Research Agent. Break down this complex query into 3 specialized subtasks for parallel execution: "{query}"

For each subtask, provide:
- Clear objective
- Specific search focus
- Expected output

SUBTASK 1: [Core/foundational aspects]
SUBTASK 2: [Recent developments/trends]
SUBTASK 3: [Applications/implications]

Make each subtask distinct to avoid overlap."""

    plan = ask_ai(delegation_prompt)
    print("  ✓ Subtasks defined and delegated")

    # Step 2: Simulate Parallel Subagents (simplified for demo)
    print("\n🔍 SUBAGENTS: Working in parallel...")

    # Extract subtasks and create targeted searches
    subtask_searches = [
        f"{query} fundamentals principles",  # Core aspects
        f"{query} latest developments",  # Recent trends
        f"{query} applications real world"    # Implementation
    ]

    subagent_results = []
    for i, search_term in enumerate(subtask_searches, 1):
        print(f"  🤖 Subagent {i}: Researching {search_term}")
        results = search_web(search_term, 2)

        sources = []
        for result in results:
            if result.text and len(result.text) > 200:
                sources.append({
                    "title": result.title,
                    "content": result.text[:300]
                })

        subagent_results.append({
            "subtask": i,
            "search_focus": search_term,
            "sources": sources
        })

    total_sources = sum(len(r["sources"]) for r in subagent_results)
    print(f"  📊 Combined: {total_sources} sources from {len(subagent_results)} agents")

    # Step 3: Lead Agent - Synthesis
    print("\n👨‍💼 LEAD AGENT: Synthesizing parallel findings...")

    # Combine all subagent findings
    synthesis_context = f"ORIGINAL QUERY: {query}\n\nSUBAGENT FINDINGS:\n"
    for result in subagent_results:
        synthesis_context += f"\nSubagent {result['subtask']} ({result['search_focus']}):\n"
        for source in result['sources'][:2]:  # Limit for brevity
            synthesis_context += f"- {source['title']}: {source['content']}...\n"

    synthesis_prompt = f"""{synthesis_context}

As the Lead Agent, synthesize these parallel findings into a comprehensive report:

EXECUTIVE SUMMARY:
[2-3 sentences covering the most important insights across all subagents]

INTEGRATED FINDINGS:
• [Key finding from foundational research]
• [Key finding from recent developments]
• [Key finding from applications research]
• [Cross-cutting insight that emerged]

RESEARCH QUALITY:
- Sources analyzed: {total_sources} across {len(subagent_results)} specialized agents
- Coverage: [How well the subtasks covered the topic]"""

    final_synthesis = ask_ai(synthesis_prompt)

    print("\n" + "=" * 50)
    print("🎯 MULTI-AGENT RESEARCH COMPLETE")
    print("=" * 50)
    print(final_synthesis)

    return {
        "query": query,
        "subagents": len(subagent_results),
        "total_sources": total_sources,
        "synthesis": final_synthesis
    }

print("✅ Anthropic multi-agent system ready!")

✅ Anthropic multi-agent system ready!


In [ ]:
# Test the Anthropic multi-agent research system
result = anthropic_multiagent_research("current climate change solutions")

print("\n" + "🤖" * 30)
print("ANTHROPIC MULTI-AGENT DEMO")
print("🤖" * 30)
print(f"Query: {result['query']}")
print(f"Subagents deployed: {result['subagents']}")
print(f"Total sources: {result['total_sources']}")
print("\n💡 Key Innovation: Parallel specialized agents + intelligent orchestration")
print("\n🎯 Try other complex topics:")
print("anthropic_multiagent_research('quantum computing commercial applications')")
print("anthropic_multiagent_research('artificial intelligence safety frameworks')")
print("anthropic_multiagent_research('renewable energy policy implementation')")

🤖 Anthropic Multi-Agent Research: current climate change solutions
--------------------------------------------------
👨‍💼 LEAD AGENT: Planning and delegating...
  ✓ Subtasks defined and delegated

🔍 SUBAGENTS: Working in parallel...
  🤖 Subagent 1: Researching current climate change solutions fundamentals principles
  🤖 Subagent 2: Researching current climate change solutions latest developments
  🤖 Subagent 3: Researching current climate change solutions applications real world
  📊 Combined: 6 sources from 3 agents

👨‍💼 LEAD AGENT: Synthesizing parallel findings...

🎯 MULTI-AGENT RESEARCH COMPLETE
**EXECUTIVE SUMMARY:**
The current climate change solutions landscape is rapidly evolving, with a focus on scaling up efforts to drive out fossil fuels and achieve net-zero emissions. Recent developments in solar-powered carbon capture and conversion technologies, as well as breakthroughs in electrochemical devices, are promising steps towards a low-carbon future. Real-world applications, su